# Natural Language Inference using Neural Networks
Adam Ek

----------------------------------

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

----------------------------------

In this lab we'll work with neural networks for natural language inference. Our task is: given a premise sentence P and hypothesis H, what entailment relationship holds between them? Is H entailed by P, contradicted by P or neutral towards P?

Given a sentence P, if H definitely describe something true given P then it is an **entailment**. If H describe something that's *maybe* true given P, it's **neutral**, and if H describe something that's definitely *false* given P it's a **contradiction**. 

In [1]:
# first we import some packages that we need
import torch
import torch.nn as nn
import torchtext
import torch.optim as optim
from torchtext.data import Field, BucketIterator, Iterator, TabularDataset
import numpy as np
# our hyperparameters (add more when/if you need them)
device = torch.device('cuda:3')

batch_size = 8
learning_rate = 0.001
epochs = 3

# 1. Data

We will explore natural language inference using neural networks on the SNLI dataset, described in [1]. The dataset can be downloaded [here](https://nlp.stanford.edu/projects/snli/). We prepared a "simplified" version, with only the relevant columns [here](https://gubox.box.com/s/idd9b9cfbks4dnhznps0gjgbnrzsvfs4).

The (simplified) data is organized as follows (tab-separated values):
* Column 1: Premise
* Column 2: Hypothesis
* Column 3: Relation

Like in the previous lab, we'll use torchtext to build a dataloader. You can essentially do the same thing as you did in the last lab, but with our new dataset. **[1 mark]**

In [2]:
def dataloader(path_to_snli):
    
    whitespacer = lambda x: x.split(' ')
    
    # "fields" that process the different columns in our CSV files
    PREMISE = Field(tokenize    = whitespacer,
                    lower       = True,
                    batch_first = True)

    HYPOTHESIS = Field(tokenize    = whitespacer,
                    lower       = True,
                    batch_first = True)

    RELATION = Field(tokenize    = whitespacer,
                    lower       = True,
                    batch_first = True)
    
    # read the csv files
    train, test = TabularDataset.splits(path   = path_to_snli,
                                        train  = 'simple_snli_1.0_train.csv',
                                        test   = 'simple_snli_1.0_test.csv',
                                        format = 'csv',
                                        fields = [('premise', PREMISE),
                                                  ('hypothesis', HYPOTHESIS),
                                                  ('relation', RELATION)],
                                        skip_header       = True,
                                        csv_reader_params = {'delimiter':'\t',
                                                             'quotechar':'Ö'})
    
    # build vocabularies based on what our csv files contained and create word2id mapping
    PREMISE.build_vocab(train)
    HYPOTHESIS.build_vocab(train)
    RELATION.build_vocab(train)
    
    # create batches from our data, and shuffle them for each epoch
    train_iter, test_iter = BucketIterator.splits((train, test),
                                                  batch_size        = batch_size,
                                                  sort_within_batch = True,
                                                  sort_key          = lambda x: len(x.premise),
                                                  shuffle           = True,
                                                  device            = device)

    return train_iter, test_iter, PREMISE.vocab, HYPOTHESIS.vocab, RELATION.vocab

In [3]:
path_to_snli = "../../../05-natural-language-inference/data"
# train_iter, test_iter = dataloader(path_to_snli)

# 2. Model

In this part, we'll build the model for predicting the relationship between H and P.

We will process each sentence using an LSTM. Then, we will construct some representation of the sentence. When we have a representation for H and P, we will combine them into one vector which we can use to predict the relationship.

We will train a model described in [2], the BiLSTM with max-pooling model. The procedure for the model is roughly:

    1) Encode the Hypothesis and the Premise using one shared bidirectional LSTM (or two different LSTMS)
    2) Perform max over the tokens in the premise and the hypothesis
    3) Combine the encoded premise and encoded hypothesis into one representation
    4) Predict the relationship 

### Creating a representation of a sentence

Let's first consider step 2 where we perform max/mean pooling. There is a function in pytorch for this, but we'll implement it from scratch. 

Let's consider the general case, what we want to do for these methods is apply some function $f$ along dimension $i$, and we want to do this for all $i$'s. As an example we consider the matrix S with size ``(N, D)`` where N is the number of words and D the number of dimensions:

$S = \begin{bmatrix}
    s_{11} & s_{12} & s_{13} & \dots  & s_{1d} \\
    s_{21} & s_{22} & s_{23} & \dots  & s_{2d} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    s_{n1} & s_{n2} & s_{n3} & \dots  & s_{nd}
\end{bmatrix}$

What we want to do is apply our function $f$ on each dimension, taking the input $s_{1d}, s_{2d}, ..., s_{nd}$ and generating the output $x_d$. 

You will implement both the max pooling method. When performing max-pooling, $max$ will be the function which selects a _maximum_ value from a vector and $x$ is the output, thus for each dimension $d$ in our output $x$ we get:

\begin{equation}
    x_d = max(s_{1d}, s_{2d}, ..., s_{nd})
\end{equation}


This operation will reduce a batch of size ``(batch_size, num_words, dimensions)`` to ``(batch_size, dimensions)`` meaning that we now have created a sentence representation based on the content of the words representations in the sentence. 

Create a function that takes as input a tensor of size ``(batch_size, num_words, dimensions)`` then performs max pooling and returns the result (the output should be of size: ```(batch_size, dimensions)```). [**4 Marks**]

In [53]:
import numpy
def pooling(input_tensor): #input size: (B, W, D)
#     x[0,:,0]
    output_tensor = []
    for batchie in range(input_tensor.shape[0]): 
        batchies = []
        for dim in range(input_tensor.shape[-1]): 
            max_val = numpy.max(input_tensor[batchie, :, dim].tolist()) #fancy indexing hell yeah ☜(ﾟヮﾟ☜)
            batchies.append(max_val)
        output_tensor.append(batchies)
        
    return torch.tensor(output_tensor).to(device) #output size: (B, D)

In [54]:
test_torch = torch.randn(3, 4, 5)
print(test_torch)

assert (torch.max(test_torch,1).values == pooling(test_torch)).all()

tensor([[[ 0.7041, -0.4714, -1.1972, -0.3028, -0.1557],
         [-0.2148, -0.1835,  0.3535, -0.1255,  0.3503],
         [-0.7582, -0.9005,  0.6706,  1.0065, -0.5077],
         [-0.0448,  0.2814, -0.3009,  0.0413,  0.2293]],

        [[ 0.5722, -1.2052, -0.1191, -0.8844, -1.1707],
         [-0.6672,  0.9663,  1.3406,  0.4889, -0.5578],
         [-2.4945,  0.5163,  0.0669,  1.1910,  0.7388],
         [ 0.6195,  1.0333,  1.0983,  0.3557, -0.8147]],

        [[-0.9675, -0.0982, -0.9503,  0.8307, -0.6160],
         [ 0.3527,  2.2793, -1.4224,  0.8534, -0.0967],
         [-0.9295, -0.8874, -1.2677, -0.7952, -1.7825],
         [ 1.1550, -0.1877,  0.3098,  0.0616,  0.7118]]])


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:3 and cpu!

### Combining sentence representations

Next, we need to combine the premise and hypothesis into one representation. We will do this by concatenating four tensors (the final size of our tensor $X$ should be ``(batch_size, 4d)`` where ``d`` is the number of dimensions that you use): 

$$X = [P; H; |P-H|; P \cdot H]$$

Here, what we do is concatenating P, H, P times H, and the absolute value of P minus H, then return the result.

Implement the function. **[2 marks]**

In [55]:
def combine_premise_and_hypothesis(hypothesis, premise):
    
    x = [premise, hypothesis, abs(premise-hypothesis), premise*hypothesis]

    output = torch.cat(x, dim=1)
    
    return output

### Creating the model

Finally, we can build the model according to the procedure given previously by using the functions we defined above. Additionaly, in the model you should use *dropout*. For efficiency purposes, it's acceptable to only train the model with either max or mean pooling. 

Implement the model [**6 marks**]

In [56]:
class SNLIModel(nn.Module):
    def __init__(self, vocab_dim, num_rels, h_dim):
        super(SNLIModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_dim, h_dim)
        self.rnn = nn.LSTM(h_dim, h_dim, batch_first=True, bidirectional=True )
        self.classifier = nn.Linear(h_dim*2, num_rels)
        self.dropout = nn.Dropout(0.2) 
        
    def forward(self, premise, hypothesis):
        #Encode the Hypothesis and the Premise using one shared bidirectional LSTM
        p = self.embeddings(premise)
        h = self.embeddings(hypothesis)
        
        p,(_, _) = self.rnn(p)
        h,(_, _) = self.rnn(h)
        
        #Perform max over the tokens in the premise and the hypothesis
        p_pooled = pooling(p)
        h_pooled = pooling(h)
        
        #Combine the encoded premise and encoded hypothesis into one representation
        ph_representation = combine_premise_and_hypothesis(h_pooled, p_pooled)
        ph_representation = self.dropout(ph_representation)
        
        #predict the relationship
        predictions = self.classifier(ph_representation)
        
        return predictions 
        
    """def forward(self, premise, hypothesis):
        #Encode the Hypothesis and the Premise using one shared bidirectional LSTM
        p = self.embeddings(premise).to(device)
        h = self.embeddings(hypothesis).to(device)
        
        #Perform max over the tokens in the premise and the hypothesis
        p_pooled = pooling(p)
        h_pooled = pooling(h)
        
        #Combine the encoded premise and encoded hypothesis into one representation
        ph_representation = combine_premise_and_hypothesis(h_pooled, p_pooled)
        
        #predict the relationship
        predictions = self.classifier(ph_representation)
        
        return predictions """

# 3. Training and testing

As before, implement the training and testing of the model. SNLI can take a very long time to train, so I suggest you only run it for one or two epochs. **[2 marks]** 

**Tip for efficiency:** *when developing your model, try training and testing the model on one batch (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [57]:
train_iter, test_iter, p_vocab, h_vocab, r_vocab = dataloader(path_to_snli)

model = SNLIModel(len(p_vocab)+len(h_vocab), len(r_vocab), 128).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.train()
for _ in range(epochs):
    for i, batch in enumerate(train_iter):
        
        output = model(batch.premise, batch.hypothesis)
        
        #calculate the loss
        loss = loss_function(output, batch.relation.view(-1))
        total_loss += loss.item()
        # print average loss for the epoch
        print(total_loss/(i+1), end='\r')
        
        # backpropagation
        loss.backward()
        
        # optimizing
        optimizer.step()
            
        # clear gradients
        optimizer.zero_grad()
        
torch.save(model, 'snli_model.pt')

RuntimeError: mat1 dim 1 must match mat2 dim 0

In [ ]:
# test model after all epochs are completed

Suggest a _baseline_ that we can compare our model against **[2 marks]**

**Your answer should go here**

Suggest some ways (other than using a baseline) in which we can analyse the models performance **[4 marks]**.

**Your answer should go here**

Suggest some ways to improve the model **[3 marks]**.

**Your answer should go here**

### Readings

[1] Samuel R. Bowman, Gabor Angeli, Christopher Potts, and Christopher D. Manning. 2015. A large annotated corpus for learning natural language inference. In Proceedings of the 2015 Conference on Empirical Methods in Natural Language Processing (EMNLP). 

[2] Conneau, A., Kiela, D., Schwenk, H., Barrault, L., & Bordes, A. (2017). Supervised learning of universal sentence representations from natural language inference data. arXiv preprint arXiv:1705.02364.